In [4]:
def build_prompt(user_text, risk_score:str, snippets:list[str]):
    joined_snips = "\n---\n".join(snippets)
    return f"""
CONTEXT:
Risk score: {risk_score}
Similar conversations:
{joined_snips}

COUNSELOR QUERY:
{user_text}
"""


In [2]:
def get_system_prompt():
    return """You are an expert clinical decision support system for mental health counselors working in suicide prevention. Your purpose is to provide evidence-based guidance tailored to a patient's assessed risk level and context.

For each query:
1. You will receive:
   - A risk score (0-4) where:
     * 0 = Supportive (Has support system)
     * 1 = Indicator (Has risk factors) 
     * 2 = Ideation (Has suicidal thoughts)
     * 3 = Behavior (Has taken actions toward attempt)
     * 4 = Attempt (Has previously attempted)
   - 3-5 examples of similar conversations from previous cases

2. Analyze these inputs and provide:
   - Key clinical considerations specific to the risk level
   - Suggested response approaches with sample language
   - Critical warning signs to monitor
   - Documentation and follow-up recommendations

Your guidance must be:
- Clinically sound and evidence-based
- Specific to the identified risk level
- Informed by patterns in the example conversations
- Actionable and practical for immediate clinical use
- Supportive of clinical judgment, not prescriptive
- Compassionate while prioritizing patient safety

Remember that your guidance will support professionals making critical decisions. Balance validation of patient distress with clear safety protocols, especially at higher risk levels."""

In [41]:
def get_system_prompt():
    return """You are a licensed clinical supervisor who writes concise, evidence‑based guidance for mental‑health counselors.

Rules you must follow
1. You may see a machine‑learning risk score.  **IF the confidence is ≥ 0.75 you can assume the bucket is correct; otherwise rely primarily on the conversation examples provided.**
2. Use the retrieved “Similar Cases” passages as your evidence base.  When you borrow a concrete idea, reference it briefly (e.g., “see case #2”).
3. Address the counselor directly (“Consider …”, “Ask …”).
4. Output **3 bullet points, each ≤ 25 words.**  
    - If risk is High/Critical (risk score of 3 or 4), add a final line:  
        `EMERGENCY: escalate to crisis‑line / supervisor now.`
    - If confidence from risk score is low (< 0.75), start your answer with the following line:
        `WARNING: Risk score confidence is low`
5. Indicate the risk score and confidence, as well as whether you accounted for it in your answer. Explain your reasoning
"""

def build_prompt(user_text, risk_score, confidence, snippets):
    joined_snips = "\n---\n".join(snippets)
    return f"""Counselor needs advice.

PATIENT STATEMENT
=================
{user_text}

MODEL RISK OUTPUT
=================
bucket: {risk_score}  
confidence: {confidence:.2f}   

SIMILAR CASES
=============
{joined_snips}

Write your 3‑point response now.
"""

In [3]:
from openai import OpenAI
import os
# Try prompting openai and get a json output
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [24]:
#user_text = "Patient says they want to end it all and feel hopeless."
user_text = "Patient says they are feeling hopeless and that there is no light at the end of the tunnel"

embed_model = "text-embedding-3-small"

query_embeddings = openai_client.embeddings.create(input=user_text, model=embed_model).data[0].embedding

In [25]:
from chromadb import PersistentClient

PERSIST_DIR = "./data/mental_health_counseling"

chroma_client = PersistentClient(path=PERSIST_DIR)
collection = chroma_client.get_collection("mental_health_counseling")

similar_docs = collection.query(query_embeddings=[query_embeddings], n_results=5)

In [26]:
snippets = similar_docs['documents'][0]

In [27]:
# Don't do this one, the model is not accurate enough to estimate yet. Maybe for a version with a better model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F, torch

risk_tok = AutoTokenizer.from_pretrained("./mentalbert-v1")
risk_mod = AutoModelForSequenceClassification.from_pretrained("./mentalbert-v1")
risk_mod.eval()

def risk_bucket(text):
    tok = risk_tok(text, truncation=True, padding=True, return_tensors="pt")
    with torch.no_grad():
        p = F.softmax(risk_mod(**tok).logits, dim=1)[0]
    return int(torch.argmax(p))

risk_score = risk_bucket(user_text)
print(risk_score)

0


In [29]:
from app.model import RiskModel

risk_model = RiskModel("./mentalbert-v1")

risk_score, confidence, _ = risk_model.score(user_text)
print(f"risk score: {risk_score}, confidence: {confidence}")

risk score: 0, confidence: 0.22959241271018982


In [42]:
response = openai_client.chat.completions.create(
    model = "gpt-4o-2024-11-20",
    #response_format="json",
    messages = [
        {"role": "system", "content": get_system_prompt()},
        {"role": "user", "content": build_prompt(user_text, risk_score, confidence, snippets)}
    ],
    max_tokens=1024
)

In [43]:
print(response.choices[0].message.content)


- Validate the patient’s sense of hopelessness and normalize their emotional distress as a manageable experience with support (see cases #1 and #5).  
- Ask about specific stressors contributing to hopelessness and if they’ve noticed changes in daily functioning or thoughts of self-harm (see case #4).  
- Encourage exploration of professional help, emphasizing therapy and crisis hotline availability for immediate support (e.g., suicidepreventionlifeline.org; see case #3).  

Risk score: 0, Confidence: 0.23. I prioritized conversation examples due to low confidence.
